In [2]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(r'C:\Users\boyu571\boyu571_Github\01_Kakaobank_SKKU_Research_23\data\foreign_news_data\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [4]:
df[df['press'] == 'newsBTC']['url'].iloc[0]

'http://www.newsbtc.com/2017/11/09/ltceur-ethusd-ichimoku-cloud-bitcoin/'

In [6]:
newsBTC = df[df['press'] == 'newsBTC']
newsBTC

,source,author,title,description,url,urlToImage,publishedAt,content,press
53,"{'id': None, 'name': 'newsBTC'}",Kiana Diva,"LTC/EUR Breaks Above Ichimoku Cloud, While Bit...","It looks like Bitcoin’s cousins, Litecoin and ...",http://www.newsbtc.com/2017/11/09/ltceur-ethus...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-09T22:35:39Z,"It looks like Bitcoin’s cousins, Litecoin and ...",newsBTC
96,"{'id': None, 'name': 'newsBTC'}",Sarah Jenn,Bitcoin Price Technical Analysis for 11/10/201...,Bitcoin price has formed a classic reversal pa...,http://www.newsbtc.com/2017/11/10/bitcoin-pric...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T03:41:32Z,Bitcoin Price Key Highlights Bitcoin price had...,newsBTC
101,"{'id': None, 'name': 'newsBTC'}",News BTC,Pundi X ICO backed by angel investors includin...,"Jakarta, Indonesia – November 8, 2017 – Pundi ...",http://www.newsbtc.com/2017/11/10/pundi-x-ico-...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T04:00:17Z,"Jakarta, Indonesia – November 8, 2017 – Pundi ...",newsBTC
107,"{'id': None, 'name': 'newsBTC'}",Aayush Jindal,Bitcoin Cash Price Technical Analysis – BCH/US...,Key Points Bitcoin cash price is gaining pace ...,http://www.newsbtc.com/2017/11/10/bitcoin-cash...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T04:08:55Z,Key Points Bitcoin cash price is gaining pace ...,newsBTC
110,"{'id': None, 'name': 'newsBTC'}",Guest Author,Here’s What Ryan Fugger of Ripple Notoriety Is...,The cryptocurrency space is nearly a decade ol...,http://www.newsbtc.com/2017/11/10/ryan-fugger-...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T04:42:10Z,The cryptocurrency space is nearly a decade ol...,newsBTC
...,...,...,...,...,...,...,...,...,...
744289,"{'id': None, 'name': 'newsBTC'}",Yuna Rin,Tron TVL Balloons To Over $15 Billion: DeFi Gr...,The Tron (TRX) DeFi ecosystem has experienced ...,http://www.newsbtc.com/news/tron-tvl-balloons-...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T09:10:11Z,The Tron (TRX) DeFi ecosystem has experienced ...,newsBTC
744308,"{'id': None, 'name': 'newsBTC'}",Scott Matherson,Crypto Analyst Reveals Which Meme Coin Will To...,Popular crypto YouTuber Jake Gagain recently v...,http://www.newsbtc.com/news/crypto-analyst-rev...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T10:30:26Z,Popular crypto YouTuber Jake Gagain recently v...,newsBTC
744353,"{'id': None, 'name': 'newsBTC'}",Opeyemi Sule,Ethereum ETF: Valkyrie Halts Purchase Of ETH F...,"Asset management firm Valkyrie, one of the fro...",http://www.newsbtc.com/news/ethereum/ethereum-...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T13:30:22Z,"Asset management firm Valkyrie, one of the fro...",newsBTC
744375,"{'id': None, 'name': 'newsBTC'}",Scott Matherson,"Crypto CEO Very Bullish On XRP Price, Sets Mak...",Following its victory against the US Securitie...,http://www.newsbtc.com/analysis/xrp/crypto-ceo...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T15:00:56Z,Following its victory against the US Securitie...,newsBTC


In [7]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

def get_contents(yahoo):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for url in tqdm(yahoo['url']):
        try:
                response = requests.get(url, headers=headers)
                soup = BeautifulSoup(response.text, "html.parser")
                content = soup.select(".content-inner")
                cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
                contents.append(cleaned_content)
        except:
                contents.append('')

    return contents

In [8]:
head = newsBTC.head()

In [10]:
# Adding the content to the DataFrame
newsBTC['content_full'] = get_contents(newsBTC)
newsBTC

100%|██████████| 23601/23601 [13:42:41<00:00,  2.09s/it]  
C:\Users\boyu571\AppData\Local\Temp\ipykernel_4456\238250536.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newsBTC['content_full'] = get_contents(newsBTC)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
53,"{'id': None, 'name': 'newsBTC'}",Kiana Diva,"LTC/EUR Breaks Above Ichimoku Cloud, While Bit...","It looks like Bitcoin’s cousins, Litecoin and ...",http://www.newsbtc.com/2017/11/09/ltceur-ethus...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-09T22:35:39Z,"It looks like Bitcoin’s cousins, Litecoin and ...",newsBTC,"\nIt looks like Bitcoin’s cousins, Litecoin an..."
96,"{'id': None, 'name': 'newsBTC'}",Sarah Jenn,Bitcoin Price Technical Analysis for 11/10/201...,Bitcoin price has formed a classic reversal pa...,http://www.newsbtc.com/2017/11/10/bitcoin-pric...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T03:41:32Z,Bitcoin Price Key Highlights Bitcoin price had...,newsBTC,\nBitcoin Price Key Highlights\n\nBitcoin pric...
101,"{'id': None, 'name': 'newsBTC'}",News BTC,Pundi X ICO backed by angel investors includin...,"Jakarta, Indonesia – November 8, 2017 – Pundi ...",http://www.newsbtc.com/2017/11/10/pundi-x-ico-...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T04:00:17Z,"Jakarta, Indonesia – November 8, 2017 – Pundi ...",newsBTC,"\nJakarta, Indonesia – November 8, 2017 – Pund..."
107,"{'id': None, 'name': 'newsBTC'}",Aayush Jindal,Bitcoin Cash Price Technical Analysis – BCH/US...,Key Points Bitcoin cash price is gaining pace ...,http://www.newsbtc.com/2017/11/10/bitcoin-cash...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T04:08:55Z,Key Points Bitcoin cash price is gaining pace ...,newsBTC,\nKey Points\n\nBitcoin cash price is gaining ...
110,"{'id': None, 'name': 'newsBTC'}",Guest Author,Here’s What Ryan Fugger of Ripple Notoriety Is...,The cryptocurrency space is nearly a decade ol...,http://www.newsbtc.com/2017/11/10/ryan-fugger-...,http://s3.amazonaws.com/main-newsbtc-images/20...,2017-11-10T04:42:10Z,The cryptocurrency space is nearly a decade ol...,newsBTC,\nThe cryptocurrency space is nearly a decade ...
...,...,...,...,...,...,...,...,...,...,...
744289,"{'id': None, 'name': 'newsBTC'}",Yuna Rin,Tron TVL Balloons To Over $15 Billion: DeFi Gr...,The Tron (TRX) DeFi ecosystem has experienced ...,http://www.newsbtc.com/news/tron-tvl-balloons-...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T09:10:11Z,The Tron (TRX) DeFi ecosystem has experienced ...,newsBTC,\nThe Tron (TRX) DeFi ecosystem has experience...
744308,"{'id': None, 'name': 'newsBTC'}",Scott Matherson,Crypto Analyst Reveals Which Meme Coin Will To...,Popular crypto YouTuber Jake Gagain recently v...,http://www.newsbtc.com/news/crypto-analyst-rev...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T10:30:26Z,Popular crypto YouTuber Jake Gagain recently v...,newsBTC,\nPopular crypto YouTuber Jake Gagain recently...
744353,"{'id': None, 'name': 'newsBTC'}",Opeyemi Sule,Ethereum ETF: Valkyrie Halts Purchase Of ETH F...,"Asset management firm Valkyrie, one of the fro...",http://www.newsbtc.com/news/ethereum/ethereum-...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T13:30:22Z,"Asset management firm Valkyrie, one of the fro...",newsBTC,"\nAsset management firm Valkyrie, one of the f..."
744375,"{'id': None, 'name': 'newsBTC'}",Scott Matherson,"Crypto CEO Very Bullish On XRP Price, Sets Mak...",Following its victory against the US Securitie...,http://www.newsbtc.com/analysis/xrp/crypto-ceo...,https://www.newsbtc.com/wp-content/uploads/202...,2023-09-30T15:00:56Z,Following its victory against the US Securitie...,newsBTC,\nFollowing its victory against the US Securit...


In [28]:
content100['content_full']

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [36]:
content100 = newsBTC[newsBTC['content_full'] == ''].iloc[199]

In [37]:
content100['content_full'] = get_contents(content100)
content100

100%|██████████| 100/100 [00:00<00:00, 5821.46it/s]


source                            {'id': None, 'name': 'newsBTC'}
author                                             Dalmas Ngetich
title           Tron Price Analysis: TRX Risk Dropping to Nint...
description     Tron prices bullish but under pressure hours a...
url             https://www.newsbtc.com/2019/02/11/tron-price-...
urlToImage      https://www.newsbtc.com/wp-content/uploads/201...
publishedAt                                  2019-02-11T17:00:22Z
content         Tron prices bullish but under pressure hours a...
press                                                     newsBTC
content_full    [, , , , , , , , , , , , , , , , , , , , , , ,...
Name: 158978, dtype: object

In [19]:
empty_rows = newsBTC[newsBTC['content_full'] == '']

empty_rows['url'].iloc[100]

'https://www.newsbtc.com/2019/02/04/crypto-researcher-waning-bitcoin-interest-twitter-screams-bearish/'

In [38]:
newsBTC.to_csv('newsBTC_4.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content